## Fetch Rewards Coding Exercise
7/21/2021
Lili Teister

**Exercise:** https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/data-modeling.html

In [441]:
import json
import pandas as pd
import gzip
from datetime import datetime
import sqlite3
import urllib
import os

In [439]:
def download_file(link, local_path):
    if os.path.exists(local_path):
        print(f"{local_path} exists.")
    else:
        urllib.request.urlretrieve(link, local_path)
        print(f"{local_path} downloaded.")
        

In [442]:
r = "https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/receipts.json.gz"
u = "https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/users.json.gz"
b = "https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/brands.json.gz"

download_file(r, "receipts.json.gz")
download_file(u, "users.json.gz")
download_file(b, "brands.json.gz")

receipts.json.gz exists.
users.json.gz exists.
brands.json.gz exists.


### Receipts data

**Receipts Step 1.** Load json into pandas dataframe(s)

In [444]:
rcpt_file = "receipts.json.gz" 
rdata = []

with gzip.open(rcpt_file, "rb") as f:
    for line in f:
        rdata.append(json.loads(line))

receipts = pd.json_normalize(rdata)
receipts.rename(columns={"_id.$oid":"id","createDate.$date":"createDate","dateScanned.$date":"dateScanned",
                         "finishedDate.$date":"finishedDate","modifyDate.$date":"modifyDate",
                        "pointsAwardedDate.$date":"pointsAwardedDate","purchaseDate.$date":"purchaseDate"},
               inplace=True)

Receipt Item List is nested - parse out into a separate dataframe.

In [334]:
item_nn = receipts[["id","rewardsReceiptItemList"]].dropna(subset=["rewardsReceiptItemList"])

itemvals = []
for index, row in item_nn.iterrows():
    idx = row["id"]
    for item in row["rewardsReceiptItemList"]:
        d = dict(item)
        d['receiptId'] = idx
        
        itemvals.append(d)
        
items = pd.json_normalize(itemvals)
items['generatedId'] = items.index

Formatting of receipts dataframe...

In [279]:
def convert_date(col):
    if pd.isna(col):
        return None
    else:
        return datetime.fromtimestamp(col/1000).strftime('%Y-%m-%d %H:%M:%S')

In [405]:
receipts['createDate'] = receipts['createDate'].apply(convert_date)
receipts['dateScanned'] = receipts['dateScanned'].apply(convert_date)
receipts['finishedDate'] = receipts['finishedDate'].apply(convert_date)
receipts['modifyDate'] = receipts['modifyDate'].apply(convert_date)
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(convert_date)
receipts['purchaseDate'] = receipts['purchaseDate'].apply(convert_date)
receipts.drop(columns=['rewardsReceiptItemList'], inplace=True)

In [339]:
receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,2021-01-02 16:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,2021-01-02 07:24:43
2,5.0,All-receipts receipt bonus,5,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,2021-01-02 16:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,2021-01-02 16:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,2021-01-02 07:25:06


In [335]:
items.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [392]:
print("Receipts:")
print(f"Records: {len(receipts)}")
print(f"Deduplicated Records: {len(receipts.drop_duplicates())}")
print(f"Deduplicated userIds: {len(receipts['id'].drop_duplicates())}")

Receipts:
Records: 1119
Deduplicated Records: 1119
Deduplicated userIds: 1119


**Receipts Step 2.** Load dataframes into SQL tables.

In [395]:
def sqlite_connection(dbname):
    c = None
    try:
        c = sqlite3.connect(dbname)
        print(f"Database {dbname} connected with sqlite version {sqlite3.version}")
    except Error as err:
        print(err)
        
    return c

conn = sqlite_connection('fetchTest.db')

Database fetchTest.db created with sqlite version 2.6.0


Create tables with appropriate keys and insert data.

In [379]:
c = conn.cursor()

tbl_receipts = """
CREATE TABLE IF NOT EXISTS receiptsFact (
    id VARCHAR(100) PRIMARY KEY,
    userId VARCHAR(100),
    createDate DATETIME,
    dateScanned DATETIME,
    finishedDate DATETIME,
    modifyDate DATETIME,
    pointsAwardedDate DATETIME,
    purchaseDate DATETIME,
    bonusPointsEarned INT,
    bonusPointsEarnedReason VARCHAR(8000),
    pointsEarned INT,
    purchasedItemCount INT,
    rewardsReceiptStatus VARCHAR(100),
    totalSpent NUMERIC
)
"""

tbl_items = """
CREATE TABLE IF NOT EXISTS receiptItemsDim (
    generatedId INT PRIMARY KEY,
    receiptId VARCHAR(100),
    barcode VARCHAR(100), 
    description VARCHAR(8000), 
    finalPrice NUMERIC, 
    itemPrice NUMERIC, 
    needsFetchReview BIT,
    partnerItemId VARCHAR(100), 
    preventTargetGapPoints BIT, 
    quantityPurchased INT,
    userFlaggedBarcode VARCHAR(100), 
    userFlaggedNewItem BIT, 
    userFlaggedPrice NUMERIC,
    userFlaggedQuantity INT, 
    needsFetchReviewReason VARCHAR(8000),
    pointsNotAwardedReason VARCHAR(8000), 
    pointsPayerId VARCHAR(100), 
    rewardsGroup VARCHAR(200),
    rewardsProductPartnerId VARCHAR(100), 
    userFlaggedDescription VARCHAR(8000),
    originalMetaBriteBarcode VARCHAR(100), 
    originalMetaBriteDescription VARCHAR(8000), 
    brandCode VARCHAR(200),
    competitorRewardsGroup VARCHAR(200), 
    discountedItemPrice NUMERIC,
    originalReceiptItemText VARCHAR(1000), 
    itemNumber INT,
    originalMetaBriteQuantityPurchased INT, 
    pointsEarned NUMERIC, 
    targetPrice NUMERIC,
    competitiveProduct BIT, 
    originalFinalPrice NUMERIC,
    originalMetaBriteItemPrice NUMERIC, 
    deleted BIT, 
    priceAfterCoupon NUMERIC,
    metabriteCampaignId VARCHAR(1000)
)
"""

c.execute('''DROP TABLE IF EXISTS receiptsFact''')
c.execute('''DROP TABLE IF EXISTS receiptItemsDim''')
c.execute(tbl_receipts)
c.execute(tbl_items)
receipts.to_sql("receiptsFact", conn, if_exists='append', index=False)
items.to_sql("receiptItemsDim", conn, if_exists='append', index=False)
conn.commit()

In [403]:
pd.read_sql_query("select distinct itemNumber from receiptItemsDim where barcode <> 4011 limit 5", conn)

,itemNumber
0,NaN
1,1.356230e+10
2,4.400000e+10
3,8.571830e+10
4,7.874211e+10


In [342]:
conn.close()

----
### User data

**Users Step 1.** Import json data into pandas dataframe.

In [357]:
user_file = "users.json.gz" 
udata = []

with gzip.open(user_file, "rb") as f:
    for line in f:
        udata.append(json.loads(line))

users = pd.json_normalize(udata)
users.rename(columns={"_id.$oid":"userId","createdDate.$date":"createdDate","lastLogin.$date":"lastLoginDate"}, inplace=True)
users['createdDate'] = users['createdDate'].apply(convert_date)
users['lastLoginDate'] = users['lastLoginDate'].apply(convert_date)
users.head()

,active,role,signUpSource,state,userId,createdDate,lastLoginDate
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 07:24:04,2021-01-03 07:25:37
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 07:24:04,2021-01-03 07:25:37
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 07:24:04,2021-01-03 07:25:37
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 07:25:30,2021-01-03 07:25:30
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 07:24:04,2021-01-03 07:25:37


There are duplicate records in the user table. It appears that they are true duplicates in which all columns are identical. Therefore, duplicates will be simply dropped.

In [361]:
print(f"Records: {len(users)}")
print(f"Deduplicated Records: {len(users.drop_duplicates())}")
print(f"Deduplicated userIds: {len(users['userId'].drop_duplicates())}")

Records: 495
Deduplicated Records: 212
Deduplicated userIds: 212


In [362]:
users.drop_duplicates(inplace=True)
users.head()

,active,role,signUpSource,state,userId,createdDate,lastLoginDate
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 07:24:04,2021-01-03 07:25:37
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 07:25:30,2021-01-03 07:25:30
6,True,consumer,Email,WI,5ff1e1e8cfcf6c399c274ad9,2021-01-03 07:25:28,2021-01-03 07:25:28
7,True,consumer,Email,WI,5ff1e1b7cfcf6c399c274a5a,2021-01-03 07:24:39,2021-01-03 07:24:39
9,True,consumer,Email,WI,5ff1e1f1cfcf6c399c274b0b,2021-01-03 07:25:37,2021-01-03 07:25:37


**Users Step 2.** Load data to sql tables

In [381]:
conn = sqlite_connection('fetchTest.db')
c = conn.cursor()

tbl_users = """
CREATE TABLE IF NOT EXISTS usersDim (
    userId VARCHAR(100) PRIMARY KEY,
    role VARCHAR(100),
    signUpSource VARCHAR(100),
    state VARCHAR(2),
    createdDate DATETIME,
    lastLoginDate DATETIME,
    active BIT
)
"""

c.execute('''DROP TABLE IF EXISTS usersDim''')
c.execute(tbl_users)
users.to_sql("usersDim", conn, if_exists='append', index=False)
conn.commit()

Database fetchTest.db created with sqlite version 2.6.0


In [382]:
pd.read_sql_query("select * from usersDim limit 5", conn)

,userId,role,signUpSource,state,createdDate,lastLoginDate,active
0,5ff1e194b6a9d73a3a9f1052,consumer,Email,WI,2021-01-03 07:24:04,2021-01-03 07:25:37,1
1,5ff1e1eacfcf6c399c274ae6,consumer,Email,WI,2021-01-03 07:25:30,2021-01-03 07:25:30,1
2,5ff1e1e8cfcf6c399c274ad9,consumer,Email,WI,2021-01-03 07:25:28,2021-01-03 07:25:28,1
3,5ff1e1b7cfcf6c399c274a5a,consumer,Email,WI,2021-01-03 07:24:39,2021-01-03 07:24:39,1
4,5ff1e1f1cfcf6c399c274b0b,consumer,Email,WI,2021-01-03 07:25:37,2021-01-03 07:25:37,1


In [383]:
conn.close()

---
### Brands data

**Brands Step 1.** Read json data into pandas dataframe.

In [408]:
brnd_file = "brands.json.gz" 
bdata = []

with gzip.open(brnd_file, "rb") as f:
    for line in f:
        bdata.append(json.loads(line))

brands = pd.json_normalize(bdata)
brands.rename(columns={"_id.$oid":"brandId","cpg.$id.$oid":"cpgId","cpg.$ref":"cpg"}, inplace=True)
brands.head()

,barcode,category,categoryCode,name,topBrand,brandId,cpgId,cpg,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


There are no duplicated records on brandId, but the barcodes field is not unique.

In [418]:
print(f"Records: {len(brands)}")
print(f"Deduplicated Records: {len(brands.drop_duplicates())}")
print(f"Deduplicated brandIds: {len(brands['brandId'].drop_duplicates())}")
print(f"Deduplicated barcodes: {len(brands['barcode'].drop_duplicates())}")

Records: 1167
Deduplicated Records: 1167
Deduplicated brandIds: 1167
Deduplicated barcodes: 1160


**Brands Step 2.** Load to SQL tables. The primary key will be brandId.

In [415]:
conn = sqlite_connection('fetchTest.db')
c = conn.cursor()

tbl_brands = """
CREATE TABLE IF NOT EXISTS brandDim (
    brandId VARCHAR(100) PRIMARY KEY,
    barcode VARCHAR(100),
    category VARCHAR(200),
    categoryCode VARCHAR(200),
    name VARCHAR(1000),
    topBrand BIT,
    cpgId VARCHAR(100),
    cpg VARCHAR(60),
    brandCode VARCHAR(100)
)
"""

c.execute('''DROP TABLE IF EXISTS brandsDim''')
c.execute(tbl_brands)
brands.to_sql("brandsDim", conn, if_exists='append', index=False)
conn.commit()

Database fetchTest.db created with sqlite version 2.6.0


In [416]:
pd.read_sql_query('''select * from brandsDim limit 5''', conn)

,barcode,category,categoryCode,name,topBrand,brandId,cpgId,cpg,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,0,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,None
1,511111519928,Beverages,BEVERAGES,Starbucks,0,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,0,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,0,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [ ]:
conn.close()

-----
## Data Modeling
Below is the link to the data model of relational tables for the provided datasets: receiptsFact, receiptItemsDim, usersDim, and brandsDim. A pdf version is also saved to the repository folder.

https://dbdesigner.page.link/CPPsQ4LKaN5MH7Fk7

**View the notebook "fetch_questions" for an analysis of the stakeholder questions.**

----